In [ ]:
%matplotlib widget
import argparse
import pybrams
import logging
import dill

In [ ]:
def main(args: argparse.Namespace) -> int:

    pybrams.enable_logging(level=logging.INFO)

    if args.enable_cache:

        pybrams.enable_cache()
    
    if args.clear_cache:

        pybrams.clear_cache()

    if args.airplane_subtraction:

        pybrams.processing.signal.enable_airplane_subtraction()
  
    if args.command == "cams":

        cams = pybrams.optical.cams.CAMS(args)
        cams.load()
        solvers = cams.solve()
        
        return solvers
        
    elif args.command == "brams":
        
        solver_data = pybrams.trajectory.extract_solver_data(args)

        solver = pybrams.trajectory.Solver(solver_data, args)
        solver.solve()

        return solver
    
    return 0

In [ ]:

trajectory_to_study = {"2025-01-31":["57", "166"], "2022-04-27":["39","49"], "2022-03-22":["92"], "2020-07-29":["79", "105", "188", "477", "532"]}

for date_to_study, trajectory_numbers in trajectory_to_study.items():

    for trajectory_number in trajectory_numbers:

        class Args():

            enable_cache = True
            clear_cache = False
            metadata_only = False
            save_json = True
            recompute_meteors = False
            airplane_subtraction = False
            velocity_model = "constant" 
            weights_pre_t0_objective = [0.030,0.059,0.111,0.200,0.333,0.500,0.667,0.800,0.889,0.941,0.970]
            plot = False

            command = "cams"
            date = date_to_study
            number = trajectory_number
            system = None

        args = Args()

        solvers = main(args)

        with open(f'Trajectory_{date_to_study}_{trajectory_number}.dill', 'wb') as fichier_dill:
            dill.dump([solver.to_dict() for solver in solvers], fichier_dill)
        
    